In [1]:
import numpy as np
import pandas as pd
import pyspark as ps

from pyspark import RDD, SparkContext, Broadcast
from pyspark.sql import SparkSession, Row, DataFrame
from pyspark.sql.types import StructType, StructField, ArrayType, IntegerType, BooleanType, DoubleType, StringType
from pyspark.sql import functions as F

from typing import Dict, Union, Optional, Tuple, List
from functools import partial

In [2]:
class Node:
    def __init__(self, event, parent=None, lchild=None, rchild=None, depth=-1, nid=-1):
        self.event = event
        self.parent = parent
        self.lchild = lchild
        self.rchild = rchild
        self.depth = depth
        self.nid = nid

In [3]:
def split(s: str) -> List[List[str]]:
    assert isinstance(s, str)
    res = []
    stack = []
    accumulator = []
    str_acc = ''
    for c in s:
        if c == '[':
            stack.append(c)
        elif c == ']':
            stack.pop()
            if len(stack) > 0:
                if len(str_acc) > 0:
                    accumulator.append(str_acc)
                    str_acc = ''
                res.append(accumulator)
                accumulator = []
        elif c == ',':
            if len(str_acc) > 0:
                accumulator.append(str_acc)
                str_acc = ''
        elif c == ' ':
            continue
        else:
            str_acc += c
            
    return res

In [4]:
def str2tree(tree_str: str, translate_dict: Dict=None) -> Node:
    elements = tree_str.split(' ')
    prev = None
    root = None
    depth = 0
    idx = 0
    for el in elements:
        if el == '-1':
            depth -= 1
            prev = prev.parent
            continue
        translated_el = el
        if not translate_dict is None:
            translated_el = translate_dict[el]
        curr = Node(translated_el, parent=prev, depth=depth, nid=idx)
        if root is None:
            root = curr
        if not prev is None:
            if prev.lchild is None:
                prev.lchild = curr
            else:
                prev.rchild = curr
        depth += 1
        prev = curr
        idx += 1
    return root

def printtree(n):
    if n is None:
        return
    print(f'{n.event},d={n.depth} | ', end='')
    printtree(n.lchild)
    printtree(n.rchild)
    print('-1 | ', end='')
    return

In [5]:
# test
tmp = str2tree('a b -1 c d e -1 f')
printtree(tmp)

a,d=0 | b,d=1 | -1 | c,d=1 | d,d=2 | e,d=3 | -1 | f,d=3 | -1 | -1 | -1 | -1 | 

In [6]:
def generate_constraint_rule(root: Node) -> List[Tuple[int, str, Union[str, int]]]:
    prev = root
    res = []
    def _rec(n: Node):
        if n is None:
            return
        res.append((n.nid, n.event, n.parent.nid if not n.parent is None else 'root'))
        _rec(n.lchild)
        _rec(n.rchild)
        return
    _rec(root)
    return res

def verify_constraint_rule(seq_row: Row,
                           rules: List[Tuple[int, str, Union[str, int]]],
                           check_exact: bool=False,
                           offset: int=1) -> bool:
    sequence = seq_row['sequence']
    constraint_dict = {}
    validity_seq = [[True for _ in range(len(sequence[y]))] for y in range(len(sequence))]
    for rule in rules:
        rid, event, constraint = rule
        if constraint == 'root':
            check = f'{event}_S0_T0' in sequence[0]
            if not check:
                return False
            constraint_dict[rid] = 0
            validity_seq[0][sequence[0].index(f'{event}_S0_T0')] = False
        else:
            min_idx = constraint_dict[constraint]
            check = False
            idx_value = -1
            for idx in range(min_idx + offset, len(sequence)):
                for index, (valid, x) in enumerate(zip(validity_seq[idx], sequence[idx])):
                    if not valid:
                        continue
                    if x.startswith(event):
                        validity_seq[idx][index] = False
                        check = True
                        break
                if check:
                    idx_value = idx
                    break
            if not check:
                return False
            constraint_dict[rid] = idx_value
    if check_exact:
        return all(all(not x for x in y) for y in validity_seq)
    return True

In [6]:
mytree = str2tree('almost_full increase -1 increase')
constr = generate_constraint_rule(mytree)
constr

[(0, 'almost_full', 'root'), (1, 'increase', 0), (2, 'increase', 0)]

In [8]:
verify_constraint_rule(Row(sequence=[['increase_S0_T0', 'full_S1_T0'], ['increase_S1_T1']]), constr)

False

In [9]:
verify_constraint_rule(Row(sequence=[['almost_full_S0_T0', 'almost_full_S1_T0'], ['increase_S1_T1']]), constr)

False

In [10]:
verify_constraint_rule(Row(sequence=[['almost_full_S0_T0', 'increase_S1_T0'], ['increase_S1_T1', 'increase_S1_T1']]), constr)

True

In [8]:
translation_dict = {
    '1': 'Snow',
    '2': 'Rain',
    '3': 'Construction',
    '4': 'Congestion',
    '5': 'Event',
    '6': 'Fog',
    '7': 'Lane-Blocked',
    '8': 'Cold',
    '9': 'Other',
    '10': 'Storm',
    '11': 'Broken-Vehicle',
    '12': 'Incident-Weather',
    '13': 'Precipitation-UNK',
    '14': 'Hail-Other',
    '15': 'Incident-Other',
    '16': 'Flow-Incident',
    '17': 'Accident',
    '-1': '-1'
}

In [9]:
schema = StructType([
    StructField('freq', IntegerType()),
    StructField('relFreq', DoubleType()),
    StructField('confidence', DoubleType()),
    StructField('sequence', StringType())
])

data_path = 'datasets/ShortLongTermTrafficIncidents/patterns_REVISED/patterns_boston_kdd_singlestep.csv'

splitter = spark.udf.register('seqparser', split, ArrayType(ArrayType(StringType())))
filter_center = spark.udf.register('filter_center', lambda it: any('S0_T0' in x for x in it[0]), 'boolean')

df = spark.read.csv(data_path,
                    header=False,
                    schema=schema) \
                .withColumn('sequence', splitter(F.col('sequence'))) \
                .filter('filter_center(sequence)')

In [105]:
tree_pattern = '1 4'
tree = str2tree(tree_pattern, translate_dict=translation_dict)
rule = generate_constraint_rule(tree)

filter_func = partial(verify_constraint_rule, rules=rule, check_exact=True)

In [106]:
printtree(tree)

Snow,d=0 | Congestion,d=1 | -1 | -1 | 

In [107]:
matching_patterns = df.rdd.filter(filter_func).toDF()
matching_patterns.show(truncate=False)

ValueError: RDD is empty

In [7]:
def find_non_matching_sequences(df: DataFrame, trees_constraints: Broadcast):
    def _match(row: Row) -> Row:
        if row['matched']:
            return Row(**row.asDict())
        check = False
        for tree_constr in trees_constraints.value:
            check = verify_constraint_rule(row, tree_constr, check_exact=True, offset=0)
            if check:
                break
        if not check:
            return Row(**row.asDict())
        res = row.asDict()
        res['matched'] = True
        return Row(**res)
    df.printSchema()
    df2 = df.withColumn('matched', F.lit(False)) \
            .rdd \
            .map(_match) \
            .toDF()
    df2.printSchema()
    
    return df2

In [ ]:
tree_df = pd.read_csv('kdd_lstw_extraction/frequent_trees_City_MSF-10_MTL-25_BO_mycopy2.csv')
trees = [str2tree(x, translation_dict) for x in tree_df['Pattern'].tolist()]
constraints = [generate_constraint_rule(t) for t in trees]
constraints_broadcast = sc.broadcast(constraints)

nonmatching_df = find_non_matching_sequences(df.filter('freq >= 10'), constraints_broadcast)

In [112]:
df.count()

1525

In [ ]:
nonmatching_df.show()

In [ ]:
df.count()

In [110]:
nonmatching_df.count()

4225

In [111]:
nonmatching_df.filter('matched').count()

1613

In [ ]:
nonmatching_df.filter('not matched').count()

In [ ]:
nonmatching_df.filter('matched is NULL').count()

In [ ]:
nonmatching_df.filter('not matched').rdd.filter(lambda it: not any(any(x.startswith('Construction') for x in y) for y in it['sequence'])).toDF().show(100, truncate=False)

In [ ]:
df.filter('freq >= 10').orderBy('freq', ascending=False).rdd.filter(lambda it: 'Rain_S0_T0' in it['sequence'][0]).toDF().show(150, truncate=False)

In [38]:
df.filter('freq >= 10').count()

4225

In [35]:
tree_df = pd.read_csv('kdd_lstw_extraction/frequent_trees_City_MSF-10_MTL-25_BO_mycopy2.csv')
trees = [str2tree(x, translation_dict) for x in tree_df['Pattern'].tolist()]
weather_events = {'Rain', 'Fog', 'Cold', 'Snow', 'Storm', 'Hail-Other', 'Hail', 'Precipitation-UNK'}
filtered_trees = [t for t in trees if not t.event in weather_events]
filtered_rules = [generate_constraint_rule(t) for t in filtered_trees]
# constraints_broadcast = sc.broadcast(constraints)
print(len(filtered_trees))

filtered_sequences = df.filter('freq >= 10') \
                .rdd \
                .filter(lambda it: not any(any(x.startswith(t) for t in weather_events for x in y) for y in it['sequence'])) \
                .collect()
print(len(filtered_sequences))

match_shortlong = [False for _ in range(len(filtered_trees))]
match_stinv = [False for _ in range(len(filtered_sequences))]

for idtree, tree_rule in enumerate(filtered_rules):
    for idseq, seq in enumerate(filtered_sequences):
        check = verify_constraint_rule(seq, tree_rule, check_exact=True, offset=0)
        if check:
            match_shortlong[idtree] = True
            match_stinv[idseq] = True
            
print(f'ShortTerm -> STInv matches:')
print(f'Total matches: {sum(match_stinv)} over {len(match_stinv)} STInv sequences ({sum(match_stinv) / len(match_stinv)})')
print(f'STInv -> ShortTerm matches:')
print(f'Total matches: {sum(match_shortlong)} over {len(match_shortlong)} trees ({sum(match_shortlong) / len(match_shortlong)})')

111
3560
ShortTerm -> STInv matches:
Total matches: 1546 over 3560 STInv sequences (0.43426966292134833)
STInv -> ShortTerm matches:
Total matches: 100 over 111 trees (0.9009009009009009)


In [34]:
df.filter('freq >= 10') \
                .rdd \
                .filter(lambda it: not any(any(x.startswith(t) for t in weather_events for x in y) for y in it['sequence'])) \
                .filter(lambda it: any(any(x.startswith('Event') for x in y) for y in it['sequence'])) \
                .toDF(df.schema) \
                .orderBy('freq', ascending=True).show(100, truncate=False)

+----+---------------------+---------------------+-----------------------------------------+
|freq|relFreq              |confidence           |sequence                                 |
+----+---------------------+---------------------+-----------------------------------------+
|11  |1.3751547049043017E-4|1.7346053772766696E-4|[[Congestion_S0_T0], [Event_S1_T2]]      |
|14  |1.7501968971509295E-4|0.01533406352683461  |[[Event_S0_T0], [Congestion_S1_T1]]      |
|15  |1.8752109612331388E-4|null                 |[[Event_S0_T0, Event_S0_T0, Event_S0_T0]]|
|15  |1.8752109612331388E-4|null                 |[[Congestion_S3_T0, Event_S0_T0]]        |
|16  |2.000225025315348E-4 |2.523062366947883E-4 |[[Congestion_S0_T0], [Event_S1_T1]]      |
|16  |2.000225025315348E-4 |0.017524644030668127 |[[Event_S0_T0], [Congestion_S2_T1]]      |
|16  |2.000225025315348E-4 |null                 |[[Congestion_S0_T0, Event_S3_T0]]        |
|23  |2.8753234738908127E-4|0.025191675794085433 |[[Event_S0_T0], [Con

In [31]:
for flag, tree in zip(match_shortlong, filtered_trees):
    if not flag:
        printtree(tree)
        print('')

Construction,d=0 | Construction,d=1 | Construction,d=2 | -1 | -1 | -1 | 
Construction,d=0 | Construction,d=1 | -1 | Congestion,d=1 | -1 | -1 | 
Construction,d=0 | Congestion,d=1 | -1 | Congestion,d=1 | -1 | -1 | 
Construction,d=0 | Flow-Incident,d=1 | -1 | -1 | 
Construction,d=0 | Accident,d=1 | -1 | -1 | 
Congestion,d=0 | Congestion,d=1 | Congestion,d=2 | Congestion,d=3 | -1 | -1 | -1 | -1 | 
Congestion,d=0 | Congestion,d=1 | Congestion,d=2 | Congestion,d=3 | -1 | -1 | -1 | Congestion,d=1 | -1 | -1 | 
Congestion,d=0 | Congestion,d=1 | Congestion,d=2 | Congestion,d=3 | -1 | -1 | -1 | Congestion,d=1 | -1 | -1 | 
Congestion,d=0 | Congestion,d=1 | Congestion,d=2 | Congestion,d=3 | -1 | -1 | -1 | Congestion,d=1 | Congestion,d=2 | -1 | -1 | -1 | 
Congestion,d=0 | Congestion,d=1 | Congestion,d=2 | Congestion,d=3 | -1 | -1 | Congestion,d=2 | -1 | -1 | -1 | 
Congestion,d=0 | Congestion,d=1 | Congestion,d=2 | Congestion,d=3 | -1 | -1 | Congestion,d=2 | -1 | -1 | Congestion,d=1 | -1 | -1 | 
Cong

In [23]:
list(map(lambda it: it['sequence'], filtered_sequences))

[[['Construction_S0_T0', 'Construction_S2_T0']],
 [['Construction_S0_T0', 'Construction_S2_T0', 'Construction_S2_T0']],
 [['Construction_S0_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0']],
 [['Construction_S0_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0']],
 [['Construction_S0_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0']],
 [['Construction_S0_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0']],
 [['Construction_S0_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0']],
 [['Construction_S0_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_T0',
   'Construction_S2_